In [1]:
import os
import pandas as pd
import re
from collections import Counter

species = 'fusobacterium_ulcerans' 

# set directories
basedir = '/'.join(os.getcwd().split('/')[:-1]) 
comparisondir = basedir+species+'/Comparison'
diamond_directory=comparisondir+'/GFs/diamond/'

In [2]:
files=os.listdir(diamond_directory)

count = 0
empty = 0
div = 0
df_gfs_l = []
for f in files:
    delos2uniprot = dict()
    delos2scores = dict()
    
    try:
        pandas_df = pd.read_csv(diamond_directory+f,sep='\t',header=None)
    except:
        empty += 1
        df_gfs_l.append({'GFs':f.split('.')[0],
                     'mapping_Dim':0,
                     'Pan':0,
                     'Pan_prots':0,
                     'Pan_prot':0,
                     'Gen':0,
                     'Gen_prots':0,
                     'Gen_prot':0,
                     'Same':False})
        continue
    
    set_genes = set()
    for r in pandas_df.index:
        pandelos_gene = pandas_df.iloc[r,0]
        set_genes.add(pandelos_gene)
        uniprot_gene = pandas_df.iloc[r,1]

        score = pandas_df.iloc[r,2]
        if pandelos_gene in delos2scores:
            if delos2scores[pandelos_gene] > score:
                continue
            else:
                delos2scores[pandelos_gene] = score
                delos2uniprot[pandelos_gene] = uniprot_gene
        else:
            delos2uniprot[pandelos_gene] = uniprot_gene
            delos2scores[pandelos_gene] = score
            
    missing_map = set_genes - set(delos2uniprot.keys()) 
    if len(missing_map) >0 :
        print('missing: ',missing_map)
    
    r = re.compile(".*PANSPECIFIC")
    matching_gene = list(filter(r.match, delos2uniprot.keys()))
    panspecific_uniprot = [delos2uniprot[u] for u in matching_gene]
 
    r = re.compile(".*GENERAL")
    unmatching_gene = list(filter(r.match, delos2uniprot.keys()))
    general_uniprot = [delos2uniprot[u] for u in unmatching_gene]
    
    if len(general_uniprot) > 0:
        max_gen = max(Counter(general_uniprot))
    else:
        max_gen = 0
    if len(panspecific_uniprot) > 0:
        max_pan = max(Counter(panspecific_uniprot))
    else:
        max_pan = 0 
     
    flag = max_pan == max_gen
    if flag:
        count += 1
    else:         
        div += 1
        
    df_gfs_l.append({'GFs':f.split('.')[0],
                     'mapping_Dim':len(set_genes),
                     'Pan':len(panspecific_uniprot),
                     'Pan_prots':panspecific_uniprot,
                     'Pan_prot':max_pan,
                     'Gen':len(general_uniprot),
                     'Gen_prots':general_uniprot,
                     'Gen_prot':max_gen,
                     'Same':flag})

res=pd.DataFrame(df_gfs_l)

print('Matching function:',count)
print('Unmatching function:', div)
print('Protein unmapped:', empty)

print('Percentage Matching:',(count*100)/(div+empty+count))
print('Percentage Other:', ((div+empty)*100)/(div+empty+count))

1523
group_5258.fa.tsv
group_5243.fa.tsv
group_5167.fa.tsv
hcaR_2.fa.tsv
group_5198.fa.tsv
group_533.fa.tsv
group_5381.fa.tsv
group_909.fa.tsv
metH_2.fa.tsv
group_1400.fa.tsv
group_5150.fa.tsv
group_83.fa.tsv
pgi.fa.tsv
group_5397.fa.tsv
group_5242.fa.tsv
bmaC.fa.tsv
group_2146.fa.tsv
group_5157.fa.tsv
patA_1.fa.tsv
group_5187.fa.tsv
group_5392.fa.tsv
group_5300.fa.tsv
group_5151.fa.tsv
group_5342.fa.tsv
group_5275.fa.tsv
patA_2.fa.tsv
group_332.fa.tsv


In [6]:
res

,GFs,mapping_Dim,Pan,Pan_prots,Pan_prot,Gen,Gen_prots,Gen_prot,Same
0,pyrK_3,29,25,"[BOBJEOOC_02670, BOBJEOOC_02670, BOBJEOOC_0222...",BOBJEOOC_02670,4,"[BOBJEOOC_02670, BOBJEOOC_02670, BOBJEOOC_0267...",BOBJEOOC_02670,True
1,group_1017,21,1,[BOBJEOOC_02846],BOBJEOOC_02846,20,"[BOBJEOOC_02846, BOBJEOOC_02846, BOBJEOOC_0284...",BOBJEOOC_02846,True
2,dag_1,28,1,[BOBJEOOC_00664],BOBJEOOC_00664,27,"[BOBJEOOC_00664, BOBJEOOC_00664, BOBJEOOC_0066...",BOBJEOOC_00664,True
3,group_3693,27,1,[BOBJEOOC_01878],BOBJEOOC_01878,26,"[BOBJEOOC_01878, BOBJEOOC_01878, BOBJEOOC_0187...",BOBJEOOC_01878,True
4,group_133,24,4,"[BOBJEOOC_02784, BOBJEOOC_02784, BOBJEOOC_0278...",BOBJEOOC_02784,20,"[BOBJEOOC_02784, BOBJEOOC_02784, BOBJEOOC_0278...",BOBJEOOC_02784,True
...,...,...,...,...,...,...,...,...,...
1518,bglA,24,1,[BOBJEOOC_01326],BOBJEOOC_01326,23,"[BOBJEOOC_01326, BOBJEOOC_01326, BOBJEOOC_0132...",BOBJEOOC_01326,True
1519,group_2391,23,11,"[BOBJEOOC_01604, BOBJEOOC_01604, BOBJEOOC_0160...",BOBJEOOC_01604,12,"[BOBJEOOC_01604, BOBJEOOC_01604, BOBJEOOC_0160...",BOBJEOOC_01604,True
1520,group_2412,26,3,"[BOBJEOOC_02792, BOBJEOOC_02792, BOBJEOOC_02792]",BOBJEOOC_02792,23,"[BOBJEOOC_02792, BOBJEOOC_02792, BOBJEOOC_0279...",BOBJEOOC_02792,True
1521,group_332,0,0,0,0,0,0,0,False


In [11]:
#Add uniprot description
refseq=basedir+'/Roary/'+species+'/prokka/'+species+'_refseq.fasta'
protacc2name = dict()
with open(refseq) as f:
    lines = f.readlines()
    for l in lines:
        if l.startswith('>'):
            l = l.split(' ')
            protein_acc = l[0].strip('>')
            name = " ".join(l[1:len(l)-2])
            protacc2name[protein_acc] = name
protacc2name[0] = 'Not mapped'

In [15]:
# Add true GF sizes (regardless of whether they were mapped in diamond)
gf_size = pd.read_csv(comparisondir+'/comparison_GF_sizes.tsv')
true_size = dict(zip(gf_size.gene, gf_size.GF_size_pandelos))
true_size_roa = dict(zip(gf_size.gene, gf_size.GF_size_Roary))

function_pan = []
true_size_l = []
true_size_r = []
function_gen = []
for r in range(0,len(res['Pan_prot'])):
    acc_pan = res.loc[r,'Pan_prot']
    acc_gen = res.loc[r,'Gen_prot']
    name = res.loc[r,'GFs']
    
    true_size_l.append(true_size[name])
    true_size_r.append(true_size_roa[name])

    function_pan.append(protacc2name[acc_pan])
    function_gen.append(protacc2name[acc_gen])
    
res['Fun_Pan']=function_pan
res['Fun_Gen']=function_gen
res['True_size']=true_size_l
res['True_size_general']=true_size_r


In [28]:
res.to_csv(comparisondir+'/function.csv')
res

,GFs,mapping_Dim,Pan,Pan_prots,Pan_prot,Gen,Gen_prots,Gen_prot,Same,Fun_Pan,Fun_Gen,True_size,True_size_general
0,pyrK_3,29,25,"[BOBJEOOC_02670, BOBJEOOC_02670, BOBJEOOC_0222...",BOBJEOOC_02670,4,"[BOBJEOOC_02670, BOBJEOOC_02670, BOBJEOOC_0267...",BOBJEOOC_02670,True,"Dihydroorotate dehydrogenase B (NAD(+)), electron","Dihydroorotate dehydrogenase B (NAD(+)), electron",29,4
1,group_1017,21,1,[BOBJEOOC_02846],BOBJEOOC_02846,20,"[BOBJEOOC_02846, BOBJEOOC_02846, BOBJEOOC_0284...",BOBJEOOC_02846,True,,,22,20
2,dag_1,28,1,[BOBJEOOC_00664],BOBJEOOC_00664,27,"[BOBJEOOC_00664, BOBJEOOC_00664, BOBJEOOC_0066...",BOBJEOOC_00664,True,,,28,27
3,group_3693,27,1,[BOBJEOOC_01878],BOBJEOOC_01878,26,"[BOBJEOOC_01878, BOBJEOOC_01878, BOBJEOOC_0187...",BOBJEOOC_01878,True,,,27,26
4,group_133,24,4,"[BOBJEOOC_02784, BOBJEOOC_02784, BOBJEOOC_0278...",BOBJEOOC_02784,20,"[BOBJEOOC_02784, BOBJEOOC_02784, BOBJEOOC_0278...",BOBJEOOC_02784,True,,,24,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518,bglA,24,1,[BOBJEOOC_01326],BOBJEOOC_01326,23,"[BOBJEOOC_01326, BOBJEOOC_01326, BOBJEOOC_0132...",BOBJEOOC_01326,True,,,24,23
1519,group_2391,23,11,"[BOBJEOOC_01604, BOBJEOOC_01604, BOBJEOOC_0160...",BOBJEOOC_01604,12,"[BOBJEOOC_01604, BOBJEOOC_01604, BOBJEOOC_0160...",BOBJEOOC_01604,True,,,23,12
1520,group_2412,26,3,"[BOBJEOOC_02792, BOBJEOOC_02792, BOBJEOOC_02792]",BOBJEOOC_02792,23,"[BOBJEOOC_02792, BOBJEOOC_02792, BOBJEOOC_0279...",BOBJEOOC_02792,True,,,26,23
1521,group_332,0,0,0,0,0,0,0,False,Not mapped,Not mapped,12,7


Bad pipe message: %s [b'illa/5.0 (X11; Ubuntu; Linux x86_64; rv:107.0) Gecko/20100101 Firefox/107.0\r\nAccept: */*\r\nAccept-Language: en-']
Bad pipe message: %s [b',en;q=0.5\r\nAccept-Encoding: gzip, deflate, br\r\nOrigin: https://www.ncbi.nlm.nih.gov\r', b'onnection', b'keep-alive\r\nSec-Fetch-Dest: empty\r\nSec-Fetch-Mode: cors\r\n']
Bad pipe message: %s [b'illa/5.0 (X11; Ubuntu; Linux x86_64; rv:107.0) Gecko/20100101 Firefox/107.0\r\nAccept: */*\r\nAccept-Language: en-']
Bad pipe message: %s [b',en;q=0.5\r\nAccept-Encoding: gzip, deflate, br\r\nOrigin: https://www.ncbi.nlm.nih.gov\r', b'onnection', b'keep-alive\r\nSec-Fetch-Dest: empty\r\nSec-Fetch-Mode: cors\r\n']
Bad pipe message: %s [b'illa/5.0 (X11; Ubuntu; Linux x86_64; rv:107.0) Gecko/20100101 Firefox/107.0\r\nAccept: */*\r\nAccept-Language: en-']
Bad pipe message: %s [b',en;q=0.5\r\nAccept-Encoding: gzip, deflate, br\r\nOrigin: https://www.ncbi.nlm.nih.gov\r', b'onnection', b'keep-alive\r\nSec-Fetch-Dest: empty\r\nSec-Fetch-

In [19]:
res[res['Same']==False]

,GFs,mapping_Dim,Pan,Pan_prots,Pan_prot,Gen,Gen_prots,Gen_prot,Same,Fun_Pan,Fun_Gen,True_size,True_size_general
12,group_5258,0,0,0,0,0,0,0,False,Not mapped,Not mapped,3,1
24,group_5469,2,2,"[BOBJEOOC_00631, BOBJEOOC_00631]",BOBJEOOC_00631,0,[],0,False,,Not mapped,2,1
69,group_5272,5,4,"[BOBJEOOC_00822, BOBJEOOC_00839, BOBJEOOC_0082...",BOBJEOOC_00839,1,[BOBJEOOC_00822],BOBJEOOC_00822,False,Aspartate,,6,1
87,group_52,20,17,"[BOBJEOOC_00258, BOBJEOOC_00258, BOBJEOOC_0025...",BOBJEOOC_00258,3,"[BOBJEOOC_00258, BOBJEOOC_00258, BOBJEOOC_02207]",BOBJEOOC_02207,False,Protein,,20,3
92,group_5253,8,8,"[BOBJEOOC_03019, BOBJEOOC_02326, BOBJEOOC_0301...",BOBJEOOC_03019,0,[],0,False,,Not mapped,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1450,patA_2,0,0,0,0,0,0,0,False,Not mapped,Not mapped,7,5
1477,yeeO_2,27,1,[BOBJEOOC_01633],BOBJEOOC_01633,26,"[BOBJEOOC_01633, BOBJEOOC_01633, BOBJEOOC_0163...",BOBJEOOC_01634,False,,putative FMN/FAD,27,26
1480,licR_1,17,0,[],0,17,"[BOBJEOOC_00689, BOBJEOOC_00689, BOBJEOOC_0068...",BOBJEOOC_00689,False,Not mapped,putative licABCH,18,17
1485,group_5287,3,3,"[BOBJEOOC_00941, BOBJEOOC_00941, BOBJEOOC_00941]",BOBJEOOC_00941,0,[],0,False,tRNA,Not mapped,3,1
